In [6]:
import cv2 as cv
import numpy as np
import tkinter as tk
from PIL import ImageTk, Image


def match_images():
    source_img = cv.imread('source.jpg')
    target_img = cv.imread('target.jpg')
    source_gray = cv.cvtColor(source_img, cv.COLOR_BGR2GRAY)
    target_gray = cv.cvtColor(target_img, cv.COLOR_BGR2GRAY)
    height, width = target_gray.shape

    orb_detector = cv.ORB_create(1000)

    keyPoints1, des1 = orb_detector.detectAndCompute(source_gray, None)
    keyPoints2, des2 = orb_detector.detectAndCompute(target_gray, None)

    matcher = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
    matches = list(matcher.match(des1, des2))
    matches.sort(key=lambda x: x.distance)

    matches = matches[:int(len(matches) * 0.9)]
    no_of_matches = len(matches)

    p1 = np.float32([keyPoints1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    p2 = np.float32([keyPoints2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
    homography, mask = cv.findHomography(p1, p2, cv.RANSAC)
    aligned_images = cv.warpPerspective(source_img, homography, (width, height))

    # Display the result in the original window
    show_matched_image(aligned_images)


def open_image():
    target_file_path = "target.jpg"  # Path to the target image
    source_file_path = "source.jpg"  # Path to the source image

    target_img = Image.open(target_file_path)
    source_img = Image.open(source_file_path)

    target_img.thumbnail((400, 400))  # Resize the target image for display
    source_img.thumbnail((400, 400))  # Resize the source image for display

    # Display the target image in the target image label
    tk_target_img = ImageTk.PhotoImage(target_img)
    original_image_label.configure(image=tk_target_img)
    original_image_label.image = tk_target_img

    # Display the source image in the source image label
    tk_source_img = ImageTk.PhotoImage(source_img)
    source_image_label.configure(image=tk_source_img)
    source_image_label.image = tk_source_img



def show_matched_image(image):
    # Convert BGR to RGB and create a PIL Image
    img = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    img = Image.fromarray(img)

    # Resize the image for display
    img.thumbnail((400, 400))

    # Display the matched image in the matched image label
    tk_img = ImageTk.PhotoImage(img)
    matched_image_label.configure(image=tk_img)
    matched_image_label.image = tk_img

    # Preserve the target image label
    original_image_label.pack(side=tk.LEFT, padx=10, pady=10)



# Create the main window
root = tk.Tk()
root.title("Image Matcher")
root.geometry("800x400")  # Set the dimensions as desired (width x height)

# Create labels to display the images
# Create labels to display the images

source_image_label = tk.Label(root)
source_image_label.pack(side=tk.LEFT, padx=10, pady=10)

original_image_label = tk.Label(root)
original_image_label.pack(side=tk.LEFT, padx=10, pady=10)

matched_image_label = tk.Label(root)
matched_image_label.pack(side=tk.LEFT, padx=10, pady=10)

# Create buttons
open_image_button = tk.Button(root, text="Open Image", command=open_image)
match_images_button = tk.Button(root, text="Match Images", command=match_images)

# Add buttons to the window
open_image_button.pack()
match_images_button.pack()

# Start the Tkinter event loop
root.mainloop()
